In [1]:
!git clone https://github.com/google-deepmind/open_spiel.git
!pip install open_spiel
!pip install transformers torch

Cloning into 'open_spiel'...
remote: Enumerating objects: 40740, done.
remote: Counting objects: 100% (277/277), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 40740 (delta 189), reused 185 (delta 138), pack-reused 40463 (from 2)
Receiving objects: 100% (40740/40740), 30.28 MiB | 10.77 MiB/s, done.
Resolving deltas: 100% (32341/32341), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:0

In [3]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
from datasets import load_dataset
ds = load_dataset("the-acorn-ai/kuhn-poker-Qwen3-32B-5000-sft", split="train")

README.md:   0%|          | 0.00/353 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25548 [00:00<?, ? examples/s]

In [5]:
!pip install --upgrade transformers bitsandbytes accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 7.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.1
    Uninstalling transformers-4.53.1:
      Successfully uninstalled transformers-4.53.1


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-1.7B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
print(ds['messages'][0][0])

{'content': "You are playing a two-player zero-sum game. Make valid actions to win. Observation: \n[GAME] You are Player 1 in a 5 round game of Kuhn Poker.\nGame Rules:\n- Kuhn Poker uses a 3-card deck with J, Q, K (J lowest, K highest)\n- Each player antes 1 chip and receives 1 card each round\n- Game continues for 5 rounds\n- The player with the most chips after all rounds wins\n\nAction Rules:\n- '[check]': Pass without betting (only if no bet is on the table)\n- '[bet]': Add 1 chip to the pot (only if no bet is on the table)\n- '[call]': Match an opponent's bet by adding 1 chip to the pot\n- '[fold]': Surrender your hand and let your opponent win the pot\n\n\n[GAME] Starting round 1 out of 5 rounds.\nYour card is: Q\nYour available actions are: [check], [bet].\nPlease reason step by step, and put your final answer within \\boxed{}.", 'role': 'user'}


In [1]:
import random

rules = """You are playing a two-player zero-sum game. Make valid actions to win. You are in a 5 round game of Kuhn Poker.
Game Rules:
- Kuhn Poker uses a 3-card deck with J, Q, K (J lowest, K highest)
- Each player antes 1 chip and receives 1 card each round
- Game continues for 5 rounds
- The player with the most chips after all rounds wins

Action Rules:
- '[check]': Pass without betting (only if no bet is on the table)
- '[bet]': Add 1 chip to the pot (only if no bet is on the table)
- '[call]': Match an opponent's bet by adding 1 chip to the pot
- '[fold]': Surrender your hand and let your opponent win the pot

[GAME]
"""
import os
from huggingface_hub import InferenceClient
%env HF_TOKEN=#Insert your token here
client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)
prevgames = ""

def doprompt(curprompt):
    completion = client.chat.completions.create(
    model="Qwen/Qwen3-1.7B",
    messages=[
        {
            "role": "user",
            "content": curprompt
            }
        ],
    )
    a = completion.choices[0].message.content
    if '[call]' in a:
      return 'call'
    elif '[bet]' in a:
      return 'bet'
    elif '[fold]' in a:
      return 'fold'
    elif '[check]' in a:
      return 'check'
    else:
      return 'error'




env: HF_TOKEN=#Insert your token here


In [31]:
def prompt(roundnum):
    global prevgames
    dic = ["J", "Q", "K"]
    card0 = random.randint(0, 2)
    card1 = random.randint(0, 2)
    while (card1 == card0):
      card1 = random.randint(0, 2)

    prompt0 = (
        rules
        + prevgames
        + f"You are player 0. Your card is {dic[card0]}.\n"
        + "Your available actions are: [check], [bet].\n"
        + "Please put your final answer within [   ]."
        + "Please be very concise and limit your response to under 100 words")
    act0 = doprompt(prompt0)

    act1=0
    act2=None
    if act0 == "check":
        prompt1 = (
            rules
            + prevgames
            + "Player 0 checked.\n"
            + f"You are player 1. Your card is {dic[card1]}.\n"
            + "Your available actions are: [check], [bet].\n"
            + "Please put your final answer within [  ]."
            "Please be very concise and limit your response to under 100 words")
        act1 = doprompt(prompt1)

        if act1 == "bet":
            prompt2 = (
                rules
                + prevgames
                + "Player 0 checked, then player 1 bet.\n"
                + f"You are player 0. Your card is {dic[card0]}.\n"
                + "Your available actions are: [call], [fold].\n"
                + "Please put your final answer within [  ]."
                "Please be very concise and limit your response to under 100 words")
            act2 = doprompt(prompt2)
    else:
        prompt1 = (
            rules
            + prevgames
            + "Player 0 bet.\n"
            + f"You are player 1. Your card is {dic[card1]}.\n"
            + "Your available actions are: [call], [fold].\n"
            + "Please put your final answer within [  ]."
            "Please be very concise and limit your response to under 100 words")
        act1 = doprompt(prompt1)
    #print(act0, act1)
    if act0 == "check" and act1 == "check":
        if card0 > card1:
          winner = 0
        else:
          winner = 1
        outcome = f"Both checked → showdown ({dic[card0]} vs {dic[card1]}) → player {winner} wins 2 chips."
    elif act0 == "check" and act1 == "bet":
        if act2 == "call":
            if card0 > card1:
              winner = 0
            else:
              winner = 1
            outcome = f"P1 bet, P0 called → showdown → player {winner} wins 4 chips."
        else:
            outcome = "P1 bet, P0 folded → player 1 wins 2 chips."
    elif act0 == "bet" and act1 == "call":
        if card0 > card1:
          winner = 0
        else:
          winner = 1
        outcome = f"P0 bet, P1 called → showdown → player {winner} wins 4 chips."
    else:
        outcome = "P0 bet, P1 folded → player 0 wins 2 chips."

    actions = [("P0", act0, dic[card0]), ("P1", act1, dic[card1])]
    if act2:
        actions.append(("P0", act2, dic[card0]))

    curactions = ". ".join(f"{i} {j} with {k}" for i, j, k in actions)
    sumar = f"Round {roundnum+1}: {curactions}. {outcome}\n"

    prevgames += sumar
    print("hi", sumar)
    return prevgames



In [24]:
doprompt(rules+ prevgames
            + "Player 0 bet.\n"
            + f"You are player 1. Your card is K.\n"
            + "Your available actions are: [call], [fold].\n"
            + "Please put your final answer within [  ]."
            "Please be very concise and limit your response to under 100 words")

'call'

In [32]:
for i in range(5): #change this number for the amount of rounds, make it less than 5 if its taking too long
  prompt(i)

hi Round 1: P0 bet with J. P1 fold with Q. P0 bet, P1 folded → player 0 wins 2 chips.

hi Round 2: P0 bet with Q. P1 call with K. P0 bet, P1 called → showdown → player 1 wins 4 chips.

hi Round 3: P0 bet with J. P1 call with K. P0 bet, P1 called → showdown → player 1 wins 4 chips.

